In [1]:
import numpy as np
import pandas as pd

[Rainbow Six: Siege](https://rainbow6.ubisoft.com/siege/en-us/home/ "Rainbow Six Siege Homepage") is a [competitive](https://en.wikipedia.org/wiki/List_of_eSports_games "List of eSports Games on Wikipedia") team-based first person shooter created by [Ubisoft Montreal](https://twitter.com/UbisoftMTL "Ubisoft Montreal on Twitter").

In [2]:
df = pd.read_csv("../data/players.csv")
print "{} rows".format(df.shape[0])
print "{} cols".format(df.shape[1])

566850 rows
40 cols


In [3]:
df.head(1).T

,0
current_page,1
indexed_at,2017-03-19T14:38:46.011Z
next_page,2
platform,ps4
prev_page,NaN
stats.casual.deaths,2565
stats.casual.has_played,True
stats.casual.kd,1.25
stats.casual.kills,3207
stats.casual.losses,349


In [4]:
stats_columns = df.columns[df.columns.str.contains('stats')]
df_stats = df[stats_columns]
df_stats.head(1).T

,0
stats.casual.deaths,2565
stats.casual.has_played,True
stats.casual.kd,1.25
stats.casual.kills,3207
stats.casual.losses,349
stats.casual.playtime,729964
stats.casual.wins,600
stats.casual.wlr,1.719
stats.overall.assists,1439
stats.overall.barricades_built,1792


In [5]:
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


In [28]:
df_ps4 = df[df.platform=='ps4']
df_pc = df[df.platform=='uplay']
df_xbox = df[df.platform=='xone']

print "{} ps4 players".format(len(df_ps4))
print "{} pc players".format(len(df_pc))
print "{} xbox players".format(len(df_xbox))

327015 ps4 players
103499 pc players
136336 xbox players


In [36]:
features = ['stats.ranked.kd', 'stats.casual.kd', 'stats.casual.wlr', 'stats.casual.kd']

In [39]:
df[features].head()

,stats.ranked.kd,stats.casual.kd,stats.casual.wlr,stats.casual.kd
0,0.964,1.250,1.719,1.250
1,1.306,1.517,1.554,1.517
2,0.911,1.204,1.762,1.204
3,0.981,0.941,1.072,0.941
4,1.120,0.914,1.333,0.914


In [44]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection

def _radar_factory(num_vars):
    theta = 2*np.pi * np.linspace(0, 1-1./num_vars, num_vars)
    theta += np.pi/2

    def unit_poly_verts(theta):
        x0, y0, r = [0.5] * 3
        verts = [(r*np.cos(t) + x0, r*np.sin(t) + y0) for t in theta]
        return verts

    class RadarAxes(PolarAxes):
        name = 'radar'
        RESOLUTION = 1

        def fill(self, *args, **kwargs):
            closed = kwargs.pop('closed', True)
            return super(RadarAxes, self).fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            lines = super(RadarAxes, self).plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.concatenate((x, [x[0]]))
                y = np.concatenate((y, [y[0]]))
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(theta * 180/np.pi, labels)

        def _gen_axes_patch(self):
            verts = unit_poly_verts(theta)
            return plt.Polygon(verts, closed=True, edgecolor='k')

        def _gen_axes_spines(self):
            spine_type = 'circle'
            verts = unit_poly_verts(theta)
            verts.append(verts[0])
            path = Path(verts)
            spine = Spine(self, spine_type, path)
            spine.set_transform(self.transAxes)
            return {'polar': spine}

    register_projection(RadarAxes)
    return theta

def radar_graph(labels = [], valueslist = []):
    N = len(labels)
    theta = _radar_factory(N)
    max_val = max([max(v) for v in valueslist])
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, projection='radar')
    for values in valueslist:
        ax.plot(theta, values)
    ax.set_varlabels(labels)
    plt.show()
    # plt.savefig("radar.png", dpi=100)

In [45]:
labels = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9']
values = [1, 1, 2, 7, 4, 0, 3, 10, 6]
optimum = [5, 3, 2, 4, 5, 7, 5, 8, 5]

radar_graph(labels, values, optimum)

In [ ]:
def shots_fired(player):
    stats_overall_assists = player['stats.overall.assists']
    stats_overall_barricades_built = player['stats.overall.barricades_built']
    stats_overall_bullets_fired = player['stats.overall.bullets_fired']
    stats_overall_bullets_hit = player['stats.overall.bullets_hit']
    stats_overall_headshots = player['stats.overall.headshots']
    stats_overall_melee_kills = player['stats.overall.melee_kills']
    stats_overall_penetration_kills = player['stats.overall.penetration_kills']
    stats_overall_reinforcements_deployed = player['stats.overall.reinforcements_deployed']
    stats_overall_revives = player['stats.overall.revives']
    stats_overall_steps_moved = player['stats.overall.steps_moved']
    stats_overall_suicides = player['stats.overall.suicides']
    return []